# Part 2: Cross-Classification to Predict Future Trips
    Select attributes for the classification, create data set for the analysis, and identify average number of trips made by each group for 2022. Create future trip production based on assumed modifications of the future households and identify number of trips produced by each zone in the future. For trip attraction, consider the same share as in 2022, with total number of trips same as future production.

In [85]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import logit
import statsmodels.api as sm
import scipy as sp


In [86]:
#### Store the url string that hosts our .csv files 
#### Copy data files to the 2022CMS folder that should be in the same folder as this notebook
urlday = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Day_2022.csv"
urlhhl = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Household_2022.csv"
urlper = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Person_2022.csv"
urlzon = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Survey_Zones.csv"
urltrp = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Trip_2022.csv"
urlveh = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Vehicle_2022.csv"

#### Read the .csv file using `pandas` libriary and store it as a `pandas` Data Frame
dday = pd.read_csv(urlday)
dhhl = pd.read_csv(urlhhl)
dper = pd.read_csv(urlper)
dzon = pd.read_csv(urlzon)
dtrp = pd.read_csv(urltrp)
dveh = pd.read_csv(urlveh)

#### Finally, let's make sure that all columns are displayed
pd.set_option('display.max_columns', None)

In [87]:
dtrp.head()

,hh_id,person_id,person_num,day_id,travel_dow,day_num,is_complete,travel_date,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_date,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_date,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight
0,22000050,2200005001,1,220000500102,5,2,1,10/14/2022,2200005001001,1,2200005001001,1,1,5,5,1,36005,1,36061,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,1,1,37,10,10,7,0,10/14/2022,13,46,20,2,10/14/2022,14,0,55,15,2,875,0.570632,1,0.000000,0,0,2,3,0,1,4,0.000000,215.724068
1,22000050,2200005001,1,220000500102,5,2,1,10/14/2022,2200005001002,1,2200005001002,2,1,5,5,1,36061,1,36005,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,37,10,1,1,1,1,0,10/14/2022,14,0,55,2,10/14/2022,17,12,43,192,4,11508,0.453004,1,5856.666667,0,0,3,2,0,4,1,0.000000,215.724068
2,22000050,2200005001,1,220000500106,2,6,1,10/18/2022,2200005001003,1,2200005001003,3,1,2,2,1,36005,1,36061,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,1,1,32,7,7,7,0,10/18/2022,18,49,23,4,10/18/2022,19,7,5,18,2,1062,0.736290,1,14.716667,0,0,2,3,0,1,4,359.540113,215.724068
3,22000050,2200005001,1,220000500106,2,6,1,10/18/2022,2200005001004,1,2200005001004,4,1,2,2,1,36061,1,36005,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,32,7,1,1,1,1,0,10/18/2022,19,21,48,4,10/18/2022,19,48,16,26,3,1588,0.274326,1,431.733333,0,0,3,2,0,4,1,359.540113,215.724068
4,22000105,2200010501,1,220001050101,1,1,1,10/17/2022,2200010501001,1,2200010501001,1,1,1,1,1,36061,1,36061,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,995,995,995,995,995,995,995,995,995,995,2,10,2,10,2,2,2,1,10/17/2022,12,4,58,2,10/17/2022,13,1,40,57,3,3402,0.485176,1,127.200000,0,0,3,3,0,2,2,861.630290,492.360166


In [88]:
dtrp.describe()

,hh_id,person_id,person_num,day_id,travel_dow,day_num,is_complete,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight
count,8.633900e+04,8.633900e+04,86339.0,8.633900e+04,86339.000000,86339.000000,86339.000000,8.633900e+04,86339.000000,8.633900e+04,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,773.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,1080.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.0,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,79530.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000,86339.000000
mean,2.208826e+07,2.208826e+09,1.0,2.208826e+11,3.918171,3.814418,79.356583,2.208826e+12,1.575244,2.208826e+12,24.292463,0.864522,3.924600,3.923094,0.908662,34663.830934,0.907585,34637.843674,40.323863,35.936981,180.474930,853.498326,962.727470,47.517182,621.673369,617.540382,621.709691,617.545130,741.818413,823.867140,980.940062,19.097439,990.021636,993.813572,993.813086,993.812993,993.813005,993.813005,993.813028,993.813468,981.199690,976.011559,980.576518,26.037963,622.751190,970.456711,992.663107,34.643278,1.251914,33.447828,1.0,290.424547,617.005224,787.263241,913.395777,961.847937,986.208399,991.255157,993.225460,994.654328,994.654490,994.654328,772.657374,48.148091,14.898516,40.214689,7.049920,5.115707,4.990421,0.082489,13.600574,28.918982,27.686109,2.712065,13.829787,29.143122,27.756970,22.764938,80.217051,1366.057494,5.629963,80.326932,204.952810,0.061780,0.006278,5.752777,5.754607,0.343993,45.587568,45.647969,371.951353,363.117997
std,4.300051e+04,4.300051e+06,0.0,4.300051e+08,1.893058,2.056645,267.920020,4.300051e+09,1.065246,4.300051e+09,19.020701,0.342235,1.899515,1.898656,0.288090,6847.211263,0.289612,6908.147006,177.129693,177.940436,366.420033,343.481505,172.930809,177.551653,481.163165,482.438761,481.187789,482.432682,432.353545,375.210293,117.343407,70.483180,70.128409,34.329669,34.343742,34.346422,34.346087,34.346087,34.345417,34.332685,116.252332,135.997115,118.857385,27.783111,477.598773,153.960352,48.020724,178.402252,0.612638,178.623669,0.0,452.20

In [89]:
dhhl.describe()

,hh_id,hh_cms_zone_gp,participation_group,num_days_complete,num_days_complete_weekday,num_days_complete_weekend,num_complete_mon,num_complete_tue,num_complete_wed,num_complete_thu,num_complete_fri,num_complete_sat,num_complete_sun,num_people,r_num_people,num_adults,num_kids,num_students,num_workers,num_vehicles,r_num_vehicles,num_trips,income_broad,primary_language,residence_type,residence_rent_own,home_county,vehicle_change,reduced_veh_1,reduced_veh_2,reduced_veh_3,reduced_veh_4,reduced_veh_5,reduced_veh_6,reduced_veh_7,reduced_veh_8,reduced_veh_997,increased_veh_1,increased_veh_2,increased_veh_3,increased_veh_4,increased_veh_5,increased_veh_6,increased_veh_7,increased_veh_8,increased_veh_997,ev_typical_charge_1,ev_typical_charge_2,ev_typical_charge_3,ev_typical_charge_4,ev_typical_charge_5,ev_typical_charge_6,ev_typical_charge_7,ev_typical_charge_997,ev_purchase,ev_purchase_barriers_1,ev_purchase_barriers_2,ev_purchase_barriers_3,ev_purchase_barriers_4,ev_purchase_barriers_5,num_bicycles,bicycle_type_1,bicycle_type_2,bicycle_type_997,bike_change,reduced_bike_1,reduced_bike_2,reduced_bike_3,reduced_bike_4,reduced_bike_5,reduced_bike_6,reduced_bike_7,reduced_bike_8,reduced_bike_997,increased_bike_1,increased_bike_2,increased_bike_3,increased_bike_4,increased_bike_5,increased_bike_6,increased_bike_7,increased_bike_8,increased_bike_9,increased_bike_997,bike_store_1,bike_store_2,bike_store_3,bike_store_4,bike_store_5,bike_store_6,bike_store_8,bike_store_9,bike_store_997,num_scooters,scooter_typical_charge_1,scooter_typical_charge_2,scooter_typical_charge_4,scooter_typical_charge_5,scooter_typical_charge_6,scooter_typical_charge_7,scooter_typical_charge_997,packages,hh_weight
count,2.966000e+03,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.00000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.00000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000,2966.000000
mean,2.208855e+07,5.337492,6.248146,4.696898,3.447741,1.249157,0.638908,0.718139,0.728928,0.729265,0.632502,0.620364,0.628793,2.321645,2.280512,1.911666,0.409980,0.196561,1.305125,0.678018,0.667566,26.813891,112.787930,46.501349,19.718476,57.109238,36054.366150,535.335469,961.804788,961.799056,961.79973,961.788941,961.788604,961.791639,961.789278,961.790290,961.794336,941.671949,941.671274,941.662846,941.665880,941.670600,941.685772,941.674309,941.672960,941.665880,988.630479,988.626433,988.626433,988.626770,988.627107,988.626096,988.626433,988.627444,546.806811,792.788604,792.797707,792.755900,792.766689,792.797370,10.149359,598.179703,597.837829,597.819960,12.136210,926.907620,926.912003,926.920094,926.901888,926.900202,926.910654,926.904585,926.920094,926.912003,903.775455,903.757586,903.758597,903.769724,903.756912,903.765341,903.780850,903.797370,903.764329,903.76298,598.026635,597.945044,597.830074,597.809508,597.842212,597.833109,597.807485,597.806136,597.817937,951.759946,966.846932,966.826028,966.820971,966.820971,966.820634,966.8206

In [90]:
# Merge household, trips and vehicle data
dht = pd.merge(dhhl, dtrp, on = 'hh_id', how = 'outer')
dhtv = pd.merge(dht, dveh, on = 'hh_id', how = 'outer')


In [91]:
dhtv.describe(include='all')

,hh_id,hh_cms_zone_gp,participation_group,signup_platform,diary_platform,num_days_complete,num_days_complete_weekday,num_days_complete_weekend,num_complete_mon,num_complete_tue,num_complete_wed,num_complete_thu,num_complete_fri,num_complete_sat,num_complete_sun,num_people,r_num_people,num_adults,num_kids,num_students,num_workers,num_vehicles,r_num_vehicles,num_trips,income_broad,primary_language,residence_type,residence_rent_own,home_county,vehicle_change,reduced_veh_1,reduced_veh_2,reduced_veh_3,reduced_veh_4,reduced_veh_5,reduced_veh_6,reduced_veh_7,reduced_veh_8,reduced_veh_997,increased_veh_1,increased_veh_2,increased_veh_3,increased_veh_4,increased_veh_5,increased_veh_6,increased_veh_7,increased_veh_8,increased_veh_997,ev_typical_charge_1,ev_typical_charge_2,ev_typical_charge_3,ev_typical_charge_4,ev_typical_charge_5,ev_typical_charge_6,ev_typical_charge_7,ev_typical_charge_997,ev_purchase,ev_purchase_barriers_1,ev_purchase_barriers_2,ev_purchase_barriers_3,ev_purchase_barriers_4,ev_purchase_barriers_5,num_bicycles,bicycle_type_1,bicycle_type_2,bicycle_type_997,bike_change,reduced_bike_1,reduced_bike_2,reduced_bike_3,reduced_bike_4,reduced_bike_5,reduced_bike_6,reduced_bike_7,reduced_bike_8,reduced_bike_997,increased_bike_1,increased_bike_2,increased_bike_3,increased_bike_4,increased_bike_5,increased_bike_6,increased_bike_7,increased_bike_8,increased_bike_9,increased_bike_997,bike_store_1,bike_store_2,bike_store_3,bike_store_4,bike_store_5,bike_store_6,bike_store_8,bike_store_9,bike_store_997,num_scooters,scooter_typical_charge_1,scooter_typical_charge_2,scooter_typical_charge_4,scooter_typical_charge_5,scooter_typical_charge_6,scooter_typical_charge_7,scooter_typical_charge_997,packages,hh_weight_x,person_id,person_num,day_id,travel_dow,day_num,is_complete,travel_date,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_date,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_date,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,vehicle_id,vehicle_num,year,fuel_type,hh_weight_y
count,1.017000e+05,101700.000000,101700.000000,101700,101700,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.00000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.000000,101700.00000

In [92]:
for i in range(len(dhtv.columns)):
    print(dhtv.columns[i])

hh_id
hh_cms_zone_gp
participation_group
signup_platform
diary_platform
num_days_complete
num_days_complete_weekday
num_days_complete_weekend
num_complete_mon
num_complete_tue
num_complete_wed
num_complete_thu
num_complete_fri
num_complete_sat
num_complete_sun
num_people
r_num_people
num_adults
num_kids
num_students
num_workers
num_vehicles
r_num_vehicles
num_trips
income_broad
primary_language
residence_type
residence_rent_own
home_county
vehicle_change
reduced_veh_1
reduced_veh_2
reduced_veh_3
reduced_veh_4
reduced_veh_5
reduced_veh_6
reduced_veh_7
reduced_veh_8
reduced_veh_997
increased_veh_1
increased_veh_2
increased_veh_3
increased_veh_4
increased_veh_5
increased_veh_6
increased_veh_7
increased_veh_8
increased_veh_997
ev_typical_charge_1
ev_typical_charge_2
ev_typical_charge_3
ev_typical_charge_4
ev_typical_charge_5
ev_typical_charge_6
ev_typical_charge_7
ev_typical_charge_997
ev_purchase
ev_purchase_barriers_1
ev_purchase_barriers_2
ev_purchase_barriers_3
ev_purchase_barriers_4
e

## Choose attributs for trip classification

For the purpose of this part, let's consider:
- Income    `income_broad`
- Vehicle number within household `vehicle_num`
- Number of adults in household (age 18 and above) `num_adults`


In [93]:
dhtv = dhtv[(dhtv['income_broad'] != 999) & (dhtv['income_broad'] != 995) & (dhtv['vehicle_num'] != 995)]

array_income = dhtv['income_broad'].unique()
array_vehicle = dhtv['vehicle_num'].dropna().unique()
array_num_adults = dhtv['num_adults'].unique()

print(array_income,array_num_adults,array_vehicle)

[3 4 2 5 1 6 7 8] [3 2 1 5 4 7 6 8] [1. 2. 3. 4. 5. 6. 7. 8.]


In [94]:
cc_current = dhtv.groupby(['income_broad','vehicle_num','num_adults']).size()
print(cc_current)

cc_hh_current = dhtv.groupby(['income_broad','vehicle_num','num_adults','hh_id']).size()
print(cc_hh_current)

cc_mean_current = dhtv.groupby(['income_broad','vehicle_num','num_adults','hh_id']).size().groupby(level=[0,1,2]).mean()
print(cc_mean_current)

income_broad  vehicle_num  num_adults
1             1.0          1             873
                           2             658
                           3             512
                           4             102
                           5              36
                                        ... 
8             1.0          5              11
              2.0          2              13
                           3              34
                           4              13
                           5              11
Length: 139, dtype: int64
income_broad  vehicle_num  num_adults  hh_id   
1             1.0          1           22008937    78
                                       22017187    54
                                       22019294    20
                                       22029287    21
                                       22036198     3
                                                   ..
8             2.0          2           22145658     9
               

In [95]:
sum(cc_mean_current)
sum(cc_mean_current)/cc_mean_current.size

31.629841356694754

In [96]:
income_type = {1:'Under $25,000',
               2:'$25,000-$49,999',
               3:'$50,000-$74,999',
               4:'$75,000-$99,999',
               5:'$100,000-$199,999',
               6:'$200,000-$249,999',
               7:'$250,000-$499,999',
               8:'$500,000 or more'}

## Calculate the average number of trips for each group


In [97]:
total_trips = 0
miss_hh = 0
included_hh = 0
size = 0
for key_inc in array_income: 
    for key_veh in array_vehicle:
        for key_adult in array_num_adults:
            try:
                print("Average number of trips made by HHs with income ", income_type[key_inc])
                print("                              number of vehicles ", key_veh)
                print("                              number of adults ", key_adult)

                
                average = cc_mean_current[key_inc, key_veh, key_adult]
                total_trips += cc_hh_current[key_inc, key_veh, key_adult].size * average
                included_hh += 1
                
                print(f"                              is {average:.2f}")
                print("\n-------------------------------------------------------------------------------")

                size += cc_hh_current[key_inc, key_veh, key_adult].size
            
            except:
                print("---------------------------Something else went wrong---------------------------\n") 
                miss_hh += 1

Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
                              number of adults  3
                              is 35.78

-------------------------------------------------------------------------------
Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
                              number of adults  2
                              is 27.10

-------------------------------------------------------------------------------
Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
                              number of adults  1
                              is 27.47

-------------------------------------------------------------------------------
Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
              

In [98]:
print ("Number of household groups with valid data: ", included_hh)
print ("Number of household groups with invalid data: ", miss_hh)

print (f"Number of trips based on available groups:  {total_trips:.2f}")
print (f"Average number of trips per group:  {cc_mean_current.mean():.2f}")
print (f"Total number of trips assuming average number of trips for missed groups: {(total_trips + miss_hh * cc_mean_current.mean()):.2f}")                

print ("Number of hoseholds accounted for: ", size)

Number of household groups with valid data:  139
Number of household groups with invalid data:  373
Number of trips based on available groups:  51718.00
Average number of trips per group:  31.63
Total number of trips assuming average number of trips for missed groups: 63515.93
Number of hoseholds accounted for:  1742


## Creating future demand

For the future demand assume that 
- 5% of household will increase one level and 5% decrease one level
- 10% of household will have an additional vehicle and 10% of household will lose one vehicle
- 15% of household will have 1 more adult and 15% will have 1 less adult

In [99]:
dhtv['future_income_broad'] = dhtv.loc[:,'income_broad']
dhtv['future_vehicle_num'] = dhtv.loc[:,'vehicle_num']
dhtv['future_num_adults'] = dhtv.loc[:,'num_adults']

In [100]:
dhtv['future_income_broad'].unique()

array([3, 4, 2, 5, 1, 6, 7, 8])

In [101]:
hh_id_array = dhtv['hh_id'].unique()
print(hh_id_array)

[22000050 22000105 22000128 ... 22145719 22145720 22145725]


In [102]:
# future demand in terms of income changes

inc_increase = pd.Series(hh_id_array).sample(frac=0.05, random_state=50664004).tolist()
print(f"The id of household with increased income: \n{inc_increase}")

inc_increase_set = set(inc_increase)

hh_id_array_wo_increase = [hh_id for hh_id in hh_id_array if hh_id not in inc_increase_set]
print(f"\nThe id of household without increased income: \n{hh_id_array_wo_increase}")

inc_decrease = pd.Series(hh_id_array_wo_increase).sample(frac=0.05/0.95, random_state=50664004).tolist()
print(f"\nThe id of household with decreased income: \n{inc_decrease}")

inc_increase_id = dhtv['hh_id'].isin(inc_increase)
inc_decrease_id = dhtv['hh_id'].isin(inc_decrease)

dhtv.loc[inc_increase_id,'future_income_broad'] = np.minimum(dhtv.loc[inc_increase_id,'future_income_broad'] + 1,8)
dhtv.loc[inc_decrease_id,'future_income_broad'] = np.maximum(dhtv.loc[inc_decrease_id,'future_income_broad'] - 1,1)

# future demand in terms of number of vehicles changes

veh_increase = pd.Series(hh_id_array).sample(frac=0.1, random_state=50664004).tolist()
print(f"\nThe id of household with increased number of vehicles: \n{veh_increase}")

veh_increase_set = set(veh_increase)
hh_id_array_wo_veh_increase = [hh_id for hh_id in hh_id_array if hh_id not in veh_increase_set]
print(f"\nThe id of household without increased number of vehicles: \n{hh_id_array_wo_veh_increase}")

veh_decrease = pd.Series(hh_id_array_wo_veh_increase).sample(frac=0.1/0.9, random_state=50664004).tolist()
print(f"\nThe id of household with decreased number of vehicles: \n{veh_decrease}")

veh_increase_id = dhtv['hh_id'].isin(veh_increase)
veh_decrease_id = dhtv['hh_id'].isin(veh_decrease)

dhtv.loc[veh_increase_id,'future_vehicle_num'] = np.minimum(dhtv.loc[veh_increase_id,'future_vehicle_num'] + 1,8)
dhtv.loc[veh_decrease_id,'future_vehicle_num'] = np.maximum(dhtv.loc[veh_decrease_id,'future_vehicle_num'] - 1,0)

# future demand in terms of number of adults changes

adu_increase = pd.Series(hh_id_array).sample(frac=0.15, random_state=50664004).tolist()
print(f"\nThe id of household with increased number of adults: \n{adu_increase}")

adu_increase_set = set(adu_increase)

hh_id_array_wo_adu_increase = [hh_id for hh_id in hh_id_array if hh_id not in adu_increase_set]
print(f"\nThe id of household without increased number of adults: \n{hh_id_array_wo_adu_increase}")

adu_decrease = pd.Series(hh_id_array_wo_adu_increase).sample(frac=0.15/0.85, random_state=50664004).tolist()
print(f"\nThe id of household with decreased number of adults: \n{adu_decrease}")

adu_increase_id = dhtv['hh_id'].isin(adu_increase)
adu_decrease_id = dhtv['hh_id'].isin(adu_decrease)

dhtv.loc[adu_increase_id, 'future_num_adults'] = np.minimum(dhtv.loc[adu_increase_id, 'future_num_adults'] + 1, 9)
dhtv.loc[adu_decrease_id, 'future_num_adults'] = np.maximum(dhtv.loc[adu_decrease_id, 'future_num_adults'] - 1, 1)

The id of household with increased income: 
[22039375, 22053107, 22025567, 22049278, 22036198, 22076983, 22134861, 22097868, 22041257, 22011619, 22145273, 22085267, 22135539, 22125957, 22132147, 22025695, 22097034, 22065252, 22122005, 22139478, 22025124, 22134013, 22119535, 22137681, 22112709, 22028836, 22138091, 22112447, 22059780, 22001440, 22144570, 22008031, 22059778, 22036782, 22133835, 22112036, 22050400, 22082648, 22091352, 22007217, 22071704, 22035733, 22078889, 22145387, 22143824, 22143823, 22087788, 22075134, 22128002, 22113387, 22059132, 22143589, 22125666, 22143630, 22048699, 22145684, 22056228, 22088411, 22057017, 22124280, 22120517, 22080073, 22004991, 22060065, 22079758, 22144420, 22139778, 22075554, 22071440, 22133735, 22048410, 22123601, 22095889, 22143927, 22145480, 22107532, 22082349, 22145081, 22099372, 22039387, 22110199, 22142095, 22145559, 22044858, 22013453, 22052809, 22093145, 22082254, 22069781, 22005666, 22145672, 22025758, 22079607, 22144764, 22077150, 22053

In [103]:
ccm_future = dhtv.groupby(['future_income_broad', 'future_vehicle_num', 'future_num_adults']).size()
print(ccm_future)

ccm_hh_future = dhtv.groupby(['future_income_broad', 'future_vehicle_num', 'future_num_adults','hh_id']).size()
print(ccm_hh_future)

ccm_hh_future_mean = ccm_hh_future.groupby(level=[0,1,2]).mean()
print(ccm_hh_future_mean)

future_income_broad  future_vehicle_num  future_num_adults
1                    0.0                 1                      27
                                         2                       2
                                         3                     197
                                         5                      36
                     1.0                 1                    1110
                                                              ... 
8                    1.0                 5                      11
                     2.0                 2                       4
                                         3                     151
                                         5                      17
                     3.0                 3                      36
Length: 174, dtype: int64
future_income_broad  future_vehicle_num  future_num_adults  hh_id   
1                    0.0                 1                  22026972      1
                                 

In [107]:
total_future_trip = 0
count_miss_hh_future = 0
count_included_hh_future = 0
size = 0

for key_inc in array_income: 
    for key_veh in array_vehicle:
        for key_adult in array_num_adults:
            try:
                print("Average number of trips made by HHs with income ", income_type[key_inc])
                print("                              number of vehicles ", key_veh)
                print("                              number of adults ", key_adult)

                average = ccm_hh_future_mean[key_inc, key_veh, key_adult]
                total_future_trip += ccm_hh_future[key_inc, key_veh, key_adult].size * average
                count_included_hh_future += 1
                size += ccm_hh_future[key_inc, key_veh, key_adult].size
            
            except:
                try:
                    count_miss_hh_future += 1
                except:
                    print("------------------------------------void--------------------------------")

Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
                              number of adults  3
Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
                              number of adults  2
Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
                              number of adults  1
Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
                              number of adults  5
Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
                              number of adults  4
Average number of trips made by HHs with income  $50,000-$74,999
                              number of vehicles  1.0
                              number

In [111]:
print(f'Number of households with valid future data: {count_included_hh_future}')
print(f'Number of households with invalid future data: {count_miss_hh_future}')
print(f'Total future trips: {total_future_trip}')
print(f'Average future trips per household: {ccm_hh_future.mean():.2f}')

Number of households with valid future data: 141
Number of households with invalid future data: 371
Total future trips: 47509.0
Average future trips per household: 29.69


In [ ]:
trip_per_zones_2022 = dtrp['d_cms_zone'].value_counts()
zone_share_2022 = trip_per_zones_2022 / trip_per_zones_2022.sum()
future_zone_trips = zone_share_2022*total_future_trip
future_zone_trips

d_cms_zone
4     11502.109441
8      6249.866480
11     4493.432794
3      4073.583514
5      3970.134412
2      3710.411135
1      2832.194292
9      2778.818958
10     2655.560454
6      2629.147917
7      2613.740604
Name: count, dtype: float64

## Cross-Classification Results Discussion

    How future number of trips will change? How you CC analysis can be improved?

1. In my scenario, the future number of trips actually decreases compared to 2022. This decline is driven by the assumptions applied to household characteristics. Because the CC model is highly sensitive to these demographic inputs, even moderate decreases in key variables result in a noticeable drop in total trip production.

2. The cross-classification approach is transparent and easy to implement, but there are several ways it can be improved.
    - Firstly, the method relies on many categories, for examples income, number of vehicles, household size. However, these categories ignore subtle variations, which can affect the amount of travel trips.
    - Secondly, the method only consider the characteristics of houshold. But, some attributes, zone density, land use, and accessibility, might also influence the travel demand.
